In [53]:
pip install --upgrade streamlit

  Using cached streamlit-1.41.1-py2.py3-none-any.whl.metadata (8.5 kB)
Using cached streamlit-1.41.1-py2.py3-none-any.whl (9.1 MB)
  Attempting uninstall: streamlit
    Found existing installation: streamlit 1.37.1
    Uninstalling streamlit-1.37.1:
      Successfully uninstalled streamlit-1.37.1
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install --upgrade pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.5 MB 5.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.5 MB 6.9 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/11.5 MB 7.1 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.5 MB 7.8 MB/s eta 0:00:01
   ----------------------- ---------------- 6.8/11.5 MB 7.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.6/11.5 MB 6.5 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.5 MB 6.1 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.5 MB 5.8 MB/s eta 0:00:01
   -------------------------------- ------- 9.4/11.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.5 MB 5.0 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.5 MB 4.7 MB/s eta 0:00:01
   ------------------------------------- -- 10.7/11.5 MB 4.6 MB/s eta 0:00:01
   

In [32]:
%%writefile app.py

import streamlit as st
import sys
import pymysql
import pandas as pd                    # importing important libraries
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pyarrow as pa
import pyarrow.parquet as pq


mydb = pymysql.connect(                # connection to sql database to execute sql queries
    host = 'localhost',
    user = 'root',
    password = 'qwertypraveen06',
    database = 'health_care_db'
)

mycursor = mydb.cursor()

def execute_query(query):             # user-defined funtion to execute the sql queries
    mycursor.execute(query) 
    result = mycursor.fetchall() 
    column_names = [desc[0] for desc in mycursor.description] 
    df = pd.DataFrame(result, columns=column_names) 
    return df

# creating the STREAMLIT Dashboard for the data

st.markdown("<h1 style = 'color : beige;'> Healthcare Organization Data Analysis Dashboard </h1>", unsafe_allow_html = True)
st.markdown("<h2 style = 'color : grey;'> Insights into Patient Admissions, Treatment Patterns, and Financial Trends </h2>", unsafe_allow_html = True)

page = st.sidebar.radio('Navigation', ['Home','Business Case Study'])
if page == 'Home':
    st.markdown("<h1 style = 'color : beige;'> Home Page </h1>", unsafe_allow_html = True)
    st.markdown("""<h2 style = 'color: grey;font-family: 'Arial', sans-serif;
        text-align: center;
        padding: 20px;'> Welcome to the Healthcare Organization Data Analysis Dashboard! Here you can find valuable insights into various aspects of our healthcare services
        </h2> """, unsafe_allow_html=True)
    
elif page == 'Business Case Study':
    st.markdown("<h1 style = 'color : beige;'> Business Case Study</h1>", unsafe_allow_html = True)

# Define a list of options for the drop-down menu

    select_box = st.selectbox('Select any category',['1. Monthly patients admission trends','2. Diagnosis Frequency Analysis','3. Bed Occupancy Analysis',
                                                    '4. Length of Stay Distribution','5. Seasonal Admission Patterns','6. Doctors Performance feedback',
                                                    '7. The highest diagnosis count in each month','8. Insights on Utilization of various test facility',
                                                    '9. Monitor facility usage to prevent overcrowding and ensure efficient operations over months.',
                                                    '10. Test Frequency and Cost Analysis','11. Billing amounts and health insurance coverage to identify financial trends',
                                                    '12. Patient Demographics and Treatment Outcomes:'])
    
    # Defining the business cases and their queries to visualise the output
    
    if select_box == "1. Monthly patients admission trends":
        st.markdown("""<h1 style = "color : red; font : italic;"> Monthly Patients Admission Trends</h1>""", unsafe_allow_html = True)

        query = ''' 
        select 
            date_format(admit_date,'%y-%m') as month_year,
            count(Patient_ID) as total_admission
        from
            health_data
        group by
            date_format(admit_date, '%y-%m')
        order by
            month_year;
        '''
    
        df = execute_query(query)
        st.write(df)

    # plotting bar chart for the following df
        
        plt.figure(figsize = (20,12))
        plt.bar(df['month_year'],df['total_admission'],color = 'red')
        plt.xlabel('Year & Month')
        plt.ylabel('Total Patient Admission')
        plt.title('Monthly patient Admission over the years')
        st.pyplot(plt)

    elif select_box == "2. Diagnosis Frequency Analysis":
        st.markdown("""<h1 style = "color : green;"> Diagnosis Frequency Analysis</h1>""", unsafe_allow_html = True)

        query = '''
        SELECT 
            diagnosis, 
        COUNT(*) AS diagnosis_count 
        FROM 
            health_data 
        GROUP BY 
            diagnosis 
        ORDER BY 
            diagnosis_count DESC 
        LIMIT 5;
        '''

        df1 = execute_query(query)
        st.write(df1)

    # plotting horizontal bar chart representation of the data acquired
        
        plt.figure(figsize = (6,5))
        plt.barh(df1['diagnosis'],df1['diagnosis_count'],color = 'green')
        plt.xlabel('Count')
        plt.ylabel('Diagnosed infections')
        plt.title('Top 5 most common Diagnosis')
        st.pyplot(plt)

    elif select_box == '3. Bed Occupancy Analysis':
        st.markdown("""<h1 style = 'color : salmon;'> Bed Occupancy Analysis </h1>""", unsafe_allow_html = True)

        query = '''
        select 
            bed_occupancy,
            COUNT(*) AS distribution_count
        from
            health_data
        group by
            bed_occupancy
        order by
            distribution_count DESC;
        '''

        df2 = execute_query(query)
        st.write(df2)

    # defining a pie chart for the data
        
        data = {
        'ICU': 1193,
        'General': 2385,
        'Private': 3579
        }

        beds = list(data.keys())
        count = list(data.values())
        
        plt.figure(figsize = (5,3))
        plt.pie(count, labels = beds, autopct = '%.2f%%',startangle = 150)
        plt.title('Distribution of Bed Occupancy')
        st.pyplot(plt)

    elif select_box == '4. Length of Stay Distribution':
        st.markdown("""<h1 style = 'color : skyblue;'> Length of Stay Distribution </h1>""", unsafe_allow_html = True)

        query = '''
        select
            admit_date,
            discharge_date,
            DATEDIFF(discharge_date, admit_date) AS stay_length
        from
            health_data
        order by 
            stay_length DESC;
        '''

        df3 = execute_query(query)

        avg_length_of_stay = df3['stay_length'].mean()
        max_length_of_stay = df3['stay_length'].max()
        
        # hist plotting
        
        plt.figure(figsize=(10, 6)) 
        plt.hist(df3['stay_length'], bins=50, color='skyblue', edgecolor='black') 
        plt.axvline(avg_length_of_stay, color='red', linestyle='dashed', linewidth=2, label=f'Average Length of Stay: {avg_length_of_stay:.2f} days') 
        plt.axvline(max_length_of_stay, color='green', linestyle='dashed', linewidth=2, label=f'Maximum Length of Stay: {max_length_of_stay:.2f} days') 
        plt.xlabel('Length of Stay (Days)') 
        plt.ylabel('Frequency') 
        plt.title('Distribution of Length of Stay for Patients') 
        plt.legend() 
        plt.tight_layout() 
        st.pyplot(plt)

    elif select_box == '5. Seasonal Admission Patterns':
        st.markdown("<h1 style = 'color : brown;'> Seasonal Admission Patterns </h1>", unsafe_allow_html = True)

        query = '''
        select 
            MONTH(admit_date) AS Monthly_admission,
            COUNT(*) AS admission_count
        from 
            health_data
        group by
            Monthly_admission
        order by
            Monthly_admission;
        '''

        df4 = execute_query(query)
        st.write(df4)

    # Line plot for the data
        
        plt.figure(figsize = (12,5))
        plt.plot(df4['Monthly_admission'],df4['admission_count'],color = 'magenta')
        plt.xlabel('Admission over months')
        plt.ylabel('Total Admission count per month')
        plt.title('Seasonal Admission Patterns')
        st.pyplot(plt)

    elif select_box == '6. Doctors Performance feedback':
        st.markdown("<h1 style = 'color : beige;'> Doctors Performance feedback </h1>", unsafe_allow_html = True)

        query = '''
        select
            Doctor,
            COUNT(*) AS patients_treated,
            ROUND(AVG(feedback), 2) AS Average_feedback
        from
            Health_data
        group by
            doctor
        order by
            patients_treated;
        '''

        df5 = execute_query(query)
        st.write(df5)

    # Bubble plot for the data
        
        plt.figure(figsize=(10, 6))
        bubble_sizes = df5['patients_treated'] * 10  # Scale bubble size for better visualization
        plt.scatter(df5['Average_feedback'], df5['Doctor'], s=bubble_sizes, alpha=0.5, color='salmon')
        plt.xlabel('Average Feedback Score')
        plt.ylabel('Doctor')
        plt.title('Doctors Performance: Patients Treated and Average Feedback Score')
        plt.grid(True)
        plt.tight_layout()
        st.pyplot(plt)

    elif select_box == '7. The highest diagnosis count in each month':
        st.markdown("<h1 style = 'color : blue;'> The highest diagnosis count in each month </h1>", unsafe_allow_html = True)

        query = '''
        SELECT
            MONTH(admit_date) AS Admission_month,
            diagnosis,
            COUNT(*) AS diagnosis_count
        FROM
            health_data
        WHERE
            diagnosis IN ('Viral infection','flu','Malaria','Typhoid','Pneumonia','Fracture')
        GROUP BY
            MONTH(admit_date), diagnosis
        ORDER BY
            Admission_month, diagnosis_count DESC;
        '''

        df6 = execute_query(query)
        st.write(df6)

        # Line plot

        for diagnosis in df6['diagnosis'].unique():
            diagnosis_data = df6[df6['diagnosis'] == diagnosis]
            plt.plot(diagnosis_data['Admission_month'], diagnosis_data['diagnosis_count'], marker='o', label=diagnosis)
        
        plt.xlabel('Month')
        plt.ylabel('Diagnosis Count')
        plt.title('Monthly Infection Diagnosis Count')
        plt.legend(title='Diagnosis')
        plt.grid(True)
        plt.tight_layout()
        st.pyplot(plt)

    elif select_box == '8. Insights on Utilization of various test facility':
        st.markdown("<h1 style = 'color : yellow;'> Insights on Utilization of various test facility </h1>", unsafe_allow_html = True)

        query = '''
        select
            test,
            COUNT(*) AS test_count
        from
            health_data
        group by
            test
        order by
            test_count DESC;
        '''

        df7 = execute_query(query)
        st.write(df7)

        # pie chart for the data

        data = {
            'Blood Test' : 2236,
            'MRI':	1789,
            'CT Scan':	1342,
            'X-Ray':	895,
            'Ultrasound':	895
        }
        
        test = list(data.keys())
        count = list(data.values())
        
        
        plt.figure(figsize = (7,5))
        plt.pie(count,labels = test, autopct = '%.2f%%', startangle = 120)
        plt.title('Insights on Utilization of various test facility')
        st.pyplot(plt)

    elif select_box == '9. Monitor facility usage to prevent overcrowding and ensure efficient operations over months.':
        st.markdown("<h2 style = 'color : brown;'> Monitor facility usage to prevent overcrowding and ensure efficient operations over months </h2>", unsafe_allow_html = True)

        query = '''
        select
            MONTH(admit_date) AS admission_month,
            COUNT(*) AS test_count
        from
            health_data
        group by
            MONTH(admit_date)
        order by
            admission_month;
        '''

        df8 = execute_query(query)
        st.write(df8)

        # Scatter plot

        plt.figure(figsize=(12, 8))
        plt.scatter(df8['admission_month'], df8['test_count'], color='red', marker='o')
        plt.xlabel('Month')
        plt.ylabel('Test Count')
        plt.title('Scatter Plot of Monthly Test Counts')
        plt.grid(True)
        plt.tight_layout()
        st.pyplot(plt)

    elif select_box == '10. Test Frequency and Cost Analysis':
        st.markdown("<h1 style = 'color : beige;'> Test Frequency and Cost Analysis </h1>", unsafe_allow_html = True)

        query = '''
        SELECT
            DATE_FORMAT(discharge_date, '%Y-%M') AS billing_month,
            test,
            SUM(`Billing Amount`) AS total_test_cost
        FROM
            health_data
        GROUP BY
            billing_month, test
        ORDER BY
            billing_month, test;
        '''
        
        df9 = execute_query(query)
        df9['total_test_cost'] = df9['total_test_cost'].astype(float)
        table = pa.Table.from_pandas(df9)
        pq.write_table(table, 'health_data.parquet')
        table

        # Stacked Bar Chart
        
        df_pivot = df9.pivot(index = 'billing_month',columns = 'test',values = 'total_test_cost').fillna(0)
        df_pivot = df_pivot.apply(pd.to_numeric, errors='coerce').fillna(0)
        
        # Plotting the stacked bar chart using matplotlib
        df_pivot.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='Paired')
        plt.xlabel('Billing Month')
        plt.ylabel('Total Test Cost')
        plt.title('Monthly Test Cost Analysis')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        st.pyplot(plt)

    elif select_box == '11. Billing amounts and health insurance coverage to identify financial trends':
        st.markdown("<h1 style = 'color : red;'> Billing amounts and health insurance coverage to identify financial trends </h1>", unsafe_allow_html = True)

        query = '''
        SELECT
            DATE_FORMAT(discharge_date, '%Y-%m') AS billing_month,
            SUM(`Billing Amount`) AS total_billing_amount,
            SUM(`Health Insurance Amount`) AS total_insurance_amount,
            SUM(`Billing Amount`) - SUM(`Health Insurance Amount`) AS out_of_pocket_amount
        FROM
            health_data
        GROUP BY
            billing_month
        ORDER BY
            billing_month;
        '''
        
        df10 = execute_query(query)
        df10['total_billing_amount'] = df10['total_billing_amount'].astype(float)
        table = pa.Table.from_pandas(df10)
        pq.write_table(table, 'health_data.parquet')
        table

        # Line plot

        plt.figure(figsize=(12, 6))
        plt.plot(df10['billing_month'], df10['total_billing_amount'], marker='o', linestyle='-', color='skyblue', label='Total Billing Amount')
        plt.plot(df10['billing_month'], df10['total_insurance_amount'], marker='o', linestyle='-', color='lightgreen', label='Total Insurance Amount')
        plt.plot(df10['billing_month'], df10['out_of_pocket_amount'], marker='o', linestyle='-', color='salmon', label='Out-of-Pocket Amount')
        plt.xlabel('Billing Month')
        plt.ylabel('Amount')
        plt.title('Monthly Billing Analysis')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        st.pyplot(plt)

    elif select_box == '12. Patient Demographics and Treatment Outcomes:':
        st.markdown("<h1 style = 'color : pink;'>  Patient Demographics and Treatment Outcomes: </h1>", unsafe_allow_html = True)

        query = '''
        SELECT
            diagnosis,
            COUNT(*) AS diagnosis_count,
            ROUND(AVG(feedback), 2) AS avg_feedback,
            AVG(`Billing Amount`) AS avg_billing_amount
        FROM
            health_data
        GROUP BY
            diagnosis
        ORDER BY
            diagnosis_count DESC;
        '''

        df11 = execute_query(query)
        df11['avg_billing_amount'] = df11['avg_billing_amount'].astype(float)
        table = pa.Table.from_pandas(df11)
        pq.write_table(table, 'health_data.parquet')
        table

        # Grouped Bar Chart
        
        fig, ax1 = plt.subplots(figsize=(10, 6))

        bar_width = 0.25
        bar_positions = np.arange(len(df11['diagnosis']))
        
        # Plot diagnosis count and average billing amount on the primary y-axis
        bar1 = ax1.bar(bar_positions - bar_width/2, df11['diagnosis_count'], bar_width, label='Diagnosis Count', color='skyblue')
        bar3 = ax1.bar(bar_positions + bar_width/2, df11['avg_billing_amount'], bar_width, label='Average Billing Amount', color='salmon')
        
        ax1.set_xlabel('Diagnosis')
        ax1.set_ylabel('Count / Billing Amount')
        ax1.set_title('Outcomes of Diagnoses: Count, Average Feedback, and Average Billing Amount')
        ax1.set_xticks(bar_positions)
        ax1.set_xticklabels(df11['diagnosis'], rotation=45, ha='right')
        ax1.legend(loc='upper left')
        ax2 = ax1.twinx()
        bar2 = ax2.plot(bar_positions, df11['avg_feedback'], marker='o', linestyle='-', color='blue', label='Average Feedback')
        ax2.set_ylabel('Average Feedback')
        ax2.legend(loc='upper right')
        
        fig.tight_layout()
        st.pyplot(plt)
    else:
        print('Validation Error')

else:
    print('Error : Close the Window')

Overwriting app.py
